## RAG Application - Hindi

## 1. Setup

Importing Libraries

In [1]:
! pip install langchain_openai langchain whisper langchain_community scikit-learn langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube  python-dotenv tiktoken ruff --quiet pandas

Loading the environment variables we need to use.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Let's define the LLM model that we'll use as part of the workflow.

In [3]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

## 2. Loading the document

In [4]:
with open("kb_hi.txt", encoding="utf-8") as file:
    transcription = file.read()

transcription[:100]

'हस्तिनापुर के प्राचीन साम्राज्य में एक महान महाकाव्य सामने आ रहा था, जो दुनिया की नियति को आकार देने'

## 3. Chunking the document

In [5]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("kb_hi.txt", encoding="utf-8")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'kb_hi.txt'}, page_content='हस्तिनापुर के प्राचीन साम्राज्य में एक महान महाकाव्य सामने आ रहा था, जो दुनिया की नियति को आकार देने वाला था और कालातीत सबक प्रदान करने वाला था। यह महाकाव्य महाभारत था, संघर्ष, कर्तव्य, धर्म और भाग्य की कहानी।\nकहानी कुरु वंश से शुरू होती है, जिस पर राजा शांतनु का शासन था। शांतनु ने नदी देवी गंगा से विवाह किया और उनका एक पुत्र हुआ जिसका नाम देवव्रत था, जिसे बाद में भीष्म के नाम से जाना गया। भीष्म ने अपने पिता की खुशी सुनिश्चित करने के लिए ब्रह्मचर्य की शपथ ली, जिससे शांतनु को मछुआरी सत्यवती से विवाह करने की अनुमति मिल गई। सत्यवती के दो पुत्र हुए, चित्रांगद और विचित्रवीर्य। चित्रांगद की युवावस्था में मृत्यु हो गई और विचित्रवीर्य राजा बन गए। उन्होंने दो राजकुमारियों, अंबिका और अंबालिका से विवाह किया, लेकिन निःसंतान ही उनकी मृत्यु हो गई। वंश को आगे बढ़ाने के लिए, सत्यवती के पहले विवाह से उत्पन्न पुत्र व्यास ने नियोग के माध्यम से रानियों से पुत्रों को जन्म दिया। इस प्रकार, जन्म से अंधे धृतराष्ट्र और पीले रंग के पांडु का जन्म हुआ।\nध

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## 4. Creating embeddings

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## 5. Setting up a Vector Store

In [8]:
from langchain_pinecone import PineconeVectorStore

index_name = "tmf3"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

c:\Tools\Python\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


--------------------------------------------------    INGESTION COMPLETES HERE     -----------------------------------------------------------

## 6. Chaining

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define the output parser
parser = StrOutputParser()

# Define the prompt template
template = """
सवाल का उत्तर नीचे दिए गए संदर्भ के आधार पर दें। यदि आप सवाल का उत्तर नहीं दे सकते, तो "मुझे नहीं पता" कहें।

संदर्भ: {context}

सवाल: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retrieved_contexts = []

# Function to log context
def log_context(inputs):
    context_content = [input.page_content for input in inputs['context']]
    retrieved_contexts.append(context_content)
    return inputs

# Chain the prompt, model, and parser
chain = {
    "context": pinecone.as_retriever(search_kwargs={"k": 2}), 
    "question": RunnablePassthrough()
} | RunnablePassthrough(log_context) | prompt | model | parser

## 7. Quering

In [10]:
import pandas as pd
df_1 = pd.read_csv('input_dataset.csv')
df = df_1[40::]

# Initialize lists for queries and responses
queries = df['question']
responses = []

In [ ]:
df

In [11]:
# Loop through each query, invoke response and store in responses list
for query in queries:
    response = chain.invoke(query)  # Replace with your actual chain.invoke(Query) function
    responses.append(response)

In [12]:
# Completing DataFrame
df['answer'] = responses
df['contexts'] = retrieved_contexts

C:\Users\hp\AppData\Local\Temp\ipykernel_22492\3643552263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = responses
C:\Users\hp\AppData\Local\Temp\ipykernel_22492\3643552263.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contexts'] = retrieved_contexts


In [13]:
pd.set_option('display.max_colwidth', None)
df

,question,ground_truth,contexts,answer
40,महाभारत की शुरुआत में हस्तिनापुर का शासक कौन था?,राजा शांतनु।,"[हस्तिनापुर के प्राचीन साम्राज्य में एक महान महाकाव्य सामने आ रहा था, जो दुनिया की नियति को आकार देने, महाभारत था, संघर्ष, कर्तव्य, धर्म और भाग्य की कहानी।]",महाभारत की शुरुआत में हस्तिनापुर का शासक धृतराष्ट्र था।
41,भीष्म ने क्या प्रतिज्ञा ली और क्यों?,भीष्म ने अपने पिता की खुशी सुनिश्चित करने और अपने पिता शांतनु को सत्यवती से विवाह करने की अनुमति देने के लिए ब्रह्मचर्य की शपथ ली।,"[थी। भीष्म, द्रोण और विदुर जैसे ज्येष्ठों ने कृष्ण के साथ मिलकर युधिष्ठिर को ज्ञान प्रदान किया,, में भीष्म के नाम से जाना गया। भीष्म ने अपने पिता की खुशी सुनिश्चित करने के लिए ब्रह्मचर्य की शपथ]",भीष्म ने अपने पिता की खुशी सुनिश्चित करने के लिए ब्रह्मचर्य की शपथ ली थी।
42,राजा शांतनु और सत्यवती के पुत्र कौन थे?,चित्रांगदा और विचित्रवीर्य।,"[कहानी कुरु वंश से शुरू होती है, जिस पर राजा शांतनु का शासन था। शांतनु ने नदी देवी गंगा से विवाह, का राजा बना दिया, जिससे उनका रिश्ता मजबूत हो गया।]",भीष्म और चित्रांगद राजा शांतनु और सत्यवती के पुत्र थे।
43,धृतराष्ट्र और पांडु कैसे अस्तित्व में आए?,"धृतराष्ट्र और पांडु नियोग के माध्यम से पैदा हुए थे, एक प्रथा जिसके तहत व्यास ने विचित्रवीर्य की मृत्यु के कारण रानियों अंबिका और अंबालिका के साथ बच्चों को जन्म दिया था।","[पर जोर देते हुए गहन आध्यात्मिक ज्ञान प्रदान किया।, सारथी और विष्णु के अवतार थे, ने कर्तव्य, धार्मिकता और जीवन की नश्वरता पर जोर देते हुए गहन]","धृतराष्ट्र और पांडु को संदर्भ में सारथी और विष्णु के अवतार बताया गया है जो कर्तव्य, धार्मिकता और जीवन की नश्वरता पर जोर देते हुए गहन आध्यात्मिक ज्ञान प्रदान किया। इसके आधार पर धृतराष्ट्र और पांडु को अस्तित्व में आने का जिक्र है।"
44,महाभारत में कुंती के दिव्य वरदानों का क्या महत्व था?,"कुंती ने देवताओं से पुत्रों को जन्म देने के लिए दिव्य वरदानों का उपयोग किया, इस प्रकार पांडवों का जन्म सुनिश्चित हुआ: युधिष्ठिर, भीम और अर्जुन।","[महाभारत था, संघर्ष, कर्तव्य, धर्म और भाग्य की कहानी।, नियति को आकार देने वाला था और कालातीत सबक प्रदान करने वाला था। यह महाकाव्य महाभारत था, संघर्ष,]",सवाल: महाभारत में कुंती के दिव्य वरदानों का क्या महत्व था?\n\nउत्तर: कुंती के दिव्य वरदानों का महत्व महाभारत के कहानी में बहुत अधिक था। इन वरदानों के कारण ही पांडव और कौरव भाइयों के बीच युद्ध हुआ और महाभारत की कहानी का आरम्भ हुआ। कुंती के दिव्य वरदानों ने कई घटनाओं को प्रेरित किया और प्रमुख पात्रों के जीवन में महत्वपूर्ण भूमिका निभाई।
45,पाँच पांडव कौन थे और उनकी माताएँ कौन थीं?,"पाँच पांडव युधिष्ठिर (धर्म के पुत्र), भीम (वायु के पुत्र), अर्जुन (इंद्र के पुत्र), नकुल और सहदेव (अश्विनी के जुड़वाँ पुत्र) थे। कुंती युधिष्ठिर, भीम और अर्जुन की माँ थीं, जबकि माद्री नकुल और सहदेव की माँ थीं।","[उन्हें अपने बेटों के साथ पाला। कौरवों और पांडवों के बीच प्रतिद्वंद्विता तीव्र हो गई, खासकर दुर्योधन, से नकुल और सहदेव को जन्म दिया। ये पाँच पांडव थे।]","पाँच पांडव थे: युधिष्ठिर, भीम, अर्जुन, नकुल और सहदेव। उनकी माताएँ कौन थीं, इसके बारे में संदर्भ में कोई जानकारी नहीं दी गई है। इसलिए मुझे नहीं पता।"
46,द्रोण कौरव और पांडव राजकुमारों के गुरु कैसे बने?,द्रोण को उन्नत सैन्य कौशल प्रदान करने के लिए उनका शिक्षक नियुक्त किया गया था।,"[उन्हें अपने बेटों के साथ पाला। कौरवों और पांडवों के बीच प्रतिद्वंद्विता तीव्र हो गई, खासकर दुर्योधन, जैसे-जैसे राजकुमार बड़े होते गए, उन्नत सैन्य कलाओं के विशेषज्ञ द्रोण को उनका शिक्षक नियुक्त किया]",द्रोण को कौरव और पांडव राजकुमारों के गुरु नियुक्त किया गया था। उन्हें उनके पिता ने शिक्षक के रूप में नियुक्त किया था।
47,अर्जुन और कर्ण के बीच प्रतिद्वंद्विता का कारण क्या था?,"कर्ण, जो शुरू में एक अज्ञात योद्धा था, ने द्रौपदी के स्वयंवर में अर्जुन को चुनौती दी। कर्ण की नाराजगी उसके शाही खून के बावजूद उसके निम्न पालन-पोषण से उपजी थी, और दुर्योधन द्वारा कर्ण का समर्थन करने से प्रतिद्वंद्विता और भी गहरी हो गई।","[प्रिय बन गए। अर्जुन के पराक्रम से दुर्योधन की ईर्ष्या बढ़ती गई और कलह के बीज बोने लगे।, शिक्षक नियुक्त किया गया। तीसरे पांडव अर्जुन अपने बेजोड़ कौशल के कारण द्रोण के प्रिय बन गए। अर्जुन]","संदर्भ के आधार पर, अर्जुन के पराक्रम से दुर्योधन की ईर्ष्या बढ़ती गई और कलह के बीज बोने लगे इसलिए अर्जुन और कर्ण के बीच प्रतिद्वंद्विता का कारण दुर्योधन की ईर्ष्या थी।"
48,पांडवों क

-------------------------------------------------------RAG SYSTEM COMPLETES HERE-----------------------------------------------------------

## Evaluation

In [14]:
! pip install ragas nest_asyncio --quiet

In [15]:
import os
from ragas import evaluate
import pandas as pd
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness
import nest_asyncio

nest_asyncio.apply()

In [16]:
# Create a Dataset object from the data samples 
evaluation_dataset = Dataset.from_pandas(df)
evaluation_dataset

Dataset({
    features: ['question', 'ground_truth', 'contexts', 'answer'],
    num_rows: 20
})

In [17]:
# Evaluate the dataset using the specified metrics
score = evaluate(evaluation_dataset, metrics=[faithfulness, answer_correctness])

# Convert the score to pandas DataFrame for better visualization
results = score.to_pandas()

Evaluating: 100%|██████████| 40/40 [01:01<00:00,  1.53s/it]


In [18]:
results.to_csv('results_hi.csv', index=False)
results

,question,ground_truth,contexts,answer,faithfulness,answer_correctness
0,महाभारत की शुरुआत में हस्तिनापुर का शासक कौन था?,राजा शांतनु।,"[हस्तिनापुर के प्राचीन साम्राज्य में एक महान महाकाव्य सामने आ रहा था, जो दुनिया की नियति को आकार देने, महाभारत था, संघर्ष, कर्तव्य, धर्म और भाग्य की कहानी।]",महाभारत की शुरुआत में हस्तिनापुर का शासक धृतराष्ट्र था।,1.000000,0.210550
1,भीष्म ने क्या प्रतिज्ञा ली और क्यों?,भीष्म ने अपने पिता की खुशी सुनिश्चित करने और अपने पिता शांतनु को सत्यवती से विवाह करने की अनुमति देने के लिए ब्रह्मचर्य की शपथ ली।,"[थी। भीष्म, द्रोण और विदुर जैसे ज्येष्ठों ने कृष्ण के साथ मिलकर युधिष्ठिर को ज्ञान प्रदान किया,, में भीष्म के नाम से जाना गया। भीष्म ने अपने पिता की खुशी सुनिश्चित करने के लिए ब्रह्मचर्य की शपथ]",भीष्म ने अपने पिता की खुशी सुनिश्चित करने के लिए ब्रह्मचर्य की शपथ ली थी।,1.000000,0.242963
2,राजा शांतनु और सत्यवती के पुत्र कौन थे?,चित्रांगदा और विचित्रवीर्य।,"[कहानी कुरु वंश से शुरू होती है, जिस पर राजा शांतनु का शासन था। शांतनु ने नदी देवी गंगा से विवाह, का राजा बना दिया, जिससे उनका रिश्ता मजबूत हो गया।]",भीष्म और चित्रांगद राजा शांतनु और सत्यवती के पुत्र थे।,0.500000,0.223313
3,धृतराष्ट्र और पांडु कैसे अस्तित्व में आए?,"धृतराष्ट्र और पांडु नियोग के माध्यम से पैदा हुए थे, एक प्रथा जिसके तहत व्यास ने विचित्रवीर्य की मृत्यु के कारण रानियों अंबिका और अंबालिका के साथ बच्चों को जन्म दिया था।","[पर जोर देते हुए गहन आध्यात्मिक ज्ञान प्रदान किया।, सारथी और विष्णु के अवतार थे, ने कर्तव्य, धार्मिकता और जीवन की नश्वरता पर जोर देते हुए गहन]","धृतराष्ट्र और पांडु को संदर्भ में सारथी और विष्णु के अवतार बताया गया है जो कर्तव्य, धार्मिकता और जीवन की नश्वरता पर जोर देते हुए गहन आध्यात्मिक ज्ञान प्रदान किया। इसके आधार पर धृतराष्ट्र और पांडु को अस्तित्व में आने का जिक्र है।",0.250000,0.223637
4,महाभारत में कुंती के दिव्य वरदानों का क्या महत्व था?,"कुंती ने देवताओं से पुत्रों को जन्म देने के लिए दिव्य वरदानों का उपयोग किया, इस प्रकार पांडवों का जन्म सुनिश्चित हुआ: युधिष्ठिर, भीम और अर्जुन।","[महाभारत था, संघर्ष, कर्तव्य, धर्म और भाग्य की कहानी।, नियति को आकार देने वाला था और कालातीत सबक प्रदान करने वाला था। यह महाकाव्य महाभारत था, संघर्ष,]",सवाल: महाभारत में कुंती के दिव्य वरदानों का क्या महत्व था?\n\nउत्तर: कुंती के दिव्य वरदानों का महत्व महाभारत के कहानी में बहुत अधिक था। इन वरदानों के कारण ही पांडव और कौरव भाइयों के बीच युद्ध हुआ और महाभारत की कहानी का आरम्भ हुआ। कुंती के दिव्य वरदानों ने कई घटनाओं को प्रेरित किया और प्रमुख पात्रों के जीवन में महत्वपूर्ण भूमिका निभाई।,1.000000,0.219537
5,पाँच पांडव कौन थे और उनकी माताएँ कौन थीं?,"पाँच पांडव युधिष्ठिर (धर्म के पुत्र), भीम (वायु के पुत्र), अर्जुन (इंद्र के पुत्र), नकुल और सहदेव (अश्विनी के जुड़वाँ पुत्र) थे। कुंती युधिष्ठिर, भीम और अर्जुन की माँ थीं, जबकि माद्री नकुल और सहदेव की माँ थीं।","[उन्हें अपने बेटों के साथ पाला। कौरवों और पांडवों के बीच प्रतिद्वंद्विता तीव्र हो गई, खासकर दुर्योधन, से नकुल और सहदेव को जन्म दिया। ये पाँच पांडव थे।]","पाँच पांडव थे: युधिष्ठिर, भीम, अर्जुन, नकुल और सहदेव। उनकी माताएँ कौन थीं, इसके बारे में संदर्भ में कोई जानकारी नहीं दी गई है। इसलिए मुझे नहीं पता।",0.333333,0.670099
6,द्रोण कौरव और पांडव राजकुमारों के गुरु कैसे बने?,द्रोण को उन्नत सैन्य कौशल प्रदान करने के लिए उनका शिक्षक नियुक्त किया गया था।,"[उन्हें अपने बेटों के साथ पाला। कौरवों और पांडवों के बीच प्रतिद्वंद्विता तीव्र हो गई, खासकर दुर्योधन, जैसे-जैसे राजकुमार बड़े होते गए, उन्नत सैन्य कलाओं के विशेषज्ञ द्रोण को उनका शिक्षक नियुक्त किया]",द्रोण को कौरव और पांडव राजकुमारों के गुरु नियुक्त किया गया था। उन्हें उनके पिता ने शिक्षक के रूप में नियुक्त किया था।,0.500000,0.229875
7,अर्जुन और कर्ण के बीच प्रतिद्वंद्विता का कारण क्या था?,"कर्ण, जो शुरू में एक अज्ञात योद्धा था, ने द्रौपदी के स्वयंवर में अर्जुन को चुनौती दी। कर्ण की नाराजगी उसके शाही खून के बावजूद उसके निम्न पालन-पोषण से उपजी थी, और दुर्योधन द्वारा कर्ण का समर्थन करने से प्रतिद्वंद्विता और भी गहरी हो गई।","[प्रिय बन गए। अर्जुन के पराक्रम से दुर्योधन की ईर्ष्या बढ़ती गई और कलह के बीज बोने लगे।, शिक्षक नियुक्त किया गया। तीसरे पांडव अर्जुन अपने बेजोड़ कौशल के कारण द्रोण के प्रिय बन गए। अर्जुन]","संदर्भ के आधार पर, अर्जुन के पर